In [1]:
import os
import getpass

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

os.environ['OPENAI_API_KEY'] = "apikeyhere"
os.environ['ACTIVELOOP_TOKEN'] = "apikeyhere"

username = "rihp" # replace with your username from app.activeloop.ai


In [2]:
embeddings = OpenAIEmbeddings(disallowed_special=())


In [3]:
!git clone https://github.com/polywrap/python-client # replace any repository of your choice 

Cloning into 'python-client'...
remote: Enumerating objects: 4557, done.
remote: Counting objects: 100% (1087/1087), done.
remote: Compressing objects: 100% (340/340), done.
remote: Total 4557 (delta 752), reused 911 (delta 732), pack-reused 3470
Receiving objects: 100% (4557/4557), 1.53 MiB | 6.41 MiB/s, done.
Resolving deltas: 100% (3043/3043), done.


In [4]:
import os
from langchain.document_loaders import TextLoader

root_dir = './python-client'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

Created a chunk of size 1599, which is longer than the specified 1000
Created a chunk of size 1026, which is longer than the specified 1000
Created a chunk of size 1952, which is longer than the specified 1000
Created a chunk of size 1107, which is longer than the specified 1000
Created a chunk of size 1952, which is longer than the specified 1000
Created a chunk of size 1952, which is longer than the specified 1000
Created a chunk of size 1026, which is longer than the specified 1000
Created a chunk of size 1952, which is longer than the specified 1000
Created a chunk of size 1952, which is longer than the specified 1000
Created a chunk of size 1952, which is longer than the specified 1000
Created a chunk of size 1026, which is longer than the specified 1000
Created a chunk of size 1158, which is longer than the specified 1000
Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of size 1952, which is longer than the specified 1000
Created a chunk of s

In [6]:
db = DeepLake(dataset_path=f"hub://{username}/python-client", embedding_function=embeddings, public=True) #dataset would be publicly available
db.add_documents(texts)

Your Deep Lake dataset has been successfully created!
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/rihp/python-client
hub://rihp/python-client loaded successfully.


Evaluating ingest: 100%|██████████| 2/2 [00:37<00:00
-

Dataset(path='hub://rihp/python-client', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape       dtype  compression
  -------   -------    -------     -------  ------- 
 embedding  generic  (1269, 1536)  float32   None   
    ids      text     (1269, 1)      str     None   
 metadata    json     (1269, 1)      str     None   
   text      text     (1269, 1)      str     None   


['4be51ee0-f24d-11ed-812a-acde48001122',
 '4be51fc6-f24d-11ed-812a-acde48001122',
 '4be51ff8-f24d-11ed-812a-acde48001122',
 '4be52016-f24d-11ed-812a-acde48001122',
 '4be52034-f24d-11ed-812a-acde48001122',
 '4be52052-f24d-11ed-812a-acde48001122',
 '4be5207a-f24d-11ed-812a-acde48001122',
 '4be520ac-f24d-11ed-812a-acde48001122',
 '4be520de-f24d-11ed-812a-acde48001122',
 '4be520fc-f24d-11ed-812a-acde48001122',
 '4be5211a-f24d-11ed-812a-acde48001122',
 '4be52138-f24d-11ed-812a-acde48001122',
 '4be52156-f24d-11ed-812a-acde48001122',
 '4be5216a-f24d-11ed-812a-acde48001122',
 '4be52188-f24d-11ed-812a-acde48001122',
 '4be521a6-f24d-11ed-812a-acde48001122',
 '4be521c4-f24d-11ed-812a-acde48001122',
 '4be521e2-f24d-11ed-812a-acde48001122',
 '4be52200-f24d-11ed-812a-acde48001122',
 '4be5221e-f24d-11ed-812a-acde48001122',
 '4be5223c-f24d-11ed-812a-acde48001122',
 '4be52282-f24d-11ed-812a-acde48001122',
 '4be522a0-f24d-11ed-812a-acde48001122',
 '4be522be-f24d-11ed-812a-acde48001122',
 '4be522dc-f24d-

In [7]:
db = DeepLake(dataset_path=f"hub://{username}/python-client", read_only=True, embedding_function=embeddings)

\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/rihp/python-client



-

hub://rihp/python-client loaded successfully.

Deep Lake Dataset in hub://rihp/python-client already exists, loading from the storage
Dataset(path='hub://rihp/python-client', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape       dtype  compression
  -------   -------    -------     -------  ------- 
 embedding  generic  (1269, 1536)  float32   None   
    ids      text     (1269, 1)      str     None   
 metadata    json     (1269, 1)      str     None   
   text      text     (1269, 1)      str     None   


In [8]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10

In [9]:
def filter(x):
    # filter based on source code
    if 'com.google' in x['text'].data()['value']:
        return False
    
    # filter based on path e.g. extension
    metadata =  x['metadata'].data()['value']
    return 'scala' in metadata['source'] or 'py' in metadata['source']

### turn on below for custom filtering
# retriever.search_kwargs['filter'] = filter

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name='gpt-3.5-turbo') # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [11]:
questions = [
    "Give me an overview of the polywrap python client. What are its capabilities?",
    "How can I call a wrapper with the polywrap python client?",
    "Can you make a polywrap python plugin with pandas?" ,  
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: Give me an overview of the polywrap python client. What are its capabilities? 

**Answer**: The Polywrap Python client is a developer tool that allows easy integration of Web3 protocols into any application. It enables the execution of WebAssembly Polywrappers on a python environment, regardless of what language this wrapper was built in. The client is built based on the functionality of the JavaScript Polywrap Client, which is currently more robust and battle-tested. The Python client is an MVP version and has implemented some of the features available on the JS stack. The Polywrap DAO will continue to improve the Python client capabilities to reach feature parity with the JS stack, while building in parallel clients for other languages like Go and Rust. To get started with the Polywrap Python client, you need to have the polywrap_client pre-requisite installed. The package contains the Polywrap client implementation, and you can learn more about it in the Polywrap Do

In [19]:
questions = [
    """Lets create a simple polywrap application with the uniswap wrapper. This is the template you gave me before
    from polywrap_client import PolywrapClient
from polywrap_core import Uri, InvokerOptions

async def get_token_price(token_address: str):
    # Instantiate the client
    client = PolywrapClient()

    # Load the Uniswap WebAssembly wrapper through a URI
    uri = Uri('wrap://ens/mainnet/uniswap-v3.polywrap.eth')

    # Configure the client to call the `getTokenPrice` method with the given argument
    options = InvokerOptions(uri=uri, method="getTokenPrice", args={"token": token_address}, encode_result=False)

    # Invoke the wrapper and return the result
    result = await client.invoke(options)
    return result.result

if __name__ == "__main__":
    # Example usage, getting the price of DAI on Uniswap
    dai_address = "0x6b175474e89094c44da98b954eedeac495271d0f"
    dai_price = await get_token_price(dai_address)
    print(f"The price of DAI on Uniswap is {dai_price}")

This code is returning the following error


    TypeError                                 Traceback (most recent call last)
Cell In[15], line 21
     18 if __name__ == "__main__":
     19     # Example usage, getting the price of DAI on Uniswap
     20     dai_address = "0x6b175474e89094c44da98b954eedeac495271d0f"
---> 21     dai_price = await get_token_price(dai_address)
     22     print(f"The price of DAI on Uniswap is {dai_price}")

Cell In[15], line 6, in get_token_price(token_address)
      4 async def get_token_price(token_address: str):
      5     # Instantiate the client
----> 6     client = PolywrapClient()
      8     # Load the Uniswap WebAssembly wrapper through a URI
      9     uri = Uri('wrap://ens/mainnet/uniswap-v3.polywrap.eth')

TypeError: PolywrapClient.__init__() missing 1 required positional argument: 'config'""",  
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: Lets create a simple polywrap application with the uniswap wrapper. This is the template you gave me before
    from polywrap_client import PolywrapClient
from polywrap_core import Uri, InvokerOptions

async def get_token_price(token_address: str):
    # Instantiate the client
    client = PolywrapClient()

    # Load the Uniswap WebAssembly wrapper through a URI
    uri = Uri('wrap://ens/mainnet/uniswap-v3.polywrap.eth')

    # Configure the client to call the `getTokenPrice` method with the given argument
    options = InvokerOptions(uri=uri, method="getTokenPrice", args={"token": token_address}, encode_result=False)

    # Invoke the wrapper and return the result
    result = await client.invoke(options)
    return result.result

if __name__ == "__main__":
    # Example usage, getting the price of DAI on Uniswap
    dai_address = "0x6b175474e89094c44da98b954eedeac495271d0f"
    dai_price = await get_token_price(dai_address)
    print(f"The price of DAI on Uniswap is

In [21]:
from polywrap_client import PolywrapClient
from polywrap_core import ClientConfig, UriResolverAggregator, FsUriResolver, SimpleFileReader

config = ClientConfig(
    resolver=UriResolverAggregator(
        [
            # Add other URI resolvers here if needed
            FsUriResolver(file_reader=SimpleFileReader())
        ]
    )
)

client = PolywrapClient(config)

async def get_token_price(token_address: str):
    # Instantiate the client

    # Load the Uniswap WebAssembly wrapper through a URI
    uri = Uri('wrap://ens/mainnet/uniswap-v3.polywrap.eth')

    # Configure the client to call the `getTokenPrice` method with the given argument
    options = InvokerOptions(uri=uri, method="getTokenPrice", args={"token": token_address}, encode_result=False)

    # Invoke the wrapper and return the result
    result = await client.invoke(options)
    return result.result

if __name__ == "__main__":
    # Example usage, getting the price of DAI on Uniswap
    dai_address = "0x6b175474e89094c44da98b954eedeac495271d0f"
    dai_price = await get_token_price(dai_address)
    print(f"The price of DAI on Uniswap is {dai_price}")

ImportError: cannot import name 'UriResolverAggregator' from 'polywrap_core' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/polywrap_core/__init__.py)